# Affinity & Solubility prediction modeling using ChemBL database: A Machine-learning based approach (Affinity Prediction Modeling) 

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO 
import pydotplus
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from IPython.display import Image  
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.decomposition import PCA


from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

1. Read sequence, fingerprint and affinity data

In [2]:
data=pd.read_csv('Combined_data_with_FP_2PCA_affinity_modeling.csv')

2. Split data for training and testing

For the models below:
Y (output) = pchembl value based category pchembl_value>5.0 active (1) and rest inactive (0)

X (Inputs) = Morgan Fingerprint vector ( compound) and reduced features based on 2D PCA (for Target)

In [53]:
AffinityModelData=pd.concat([data.filter(like='fp'),data.filter(regex='^[0-41]'),data['pchembl_value']],axis=1)
#AffinityModelData=pd.concat([data.filter(regex='^[0-41]'),data['pchembl_value']],axis=1)
AffinityModelData=AffinityModelData.dropna()
AffinityModelData['pchembl_cat']=0 # initialize pchembl value based category
id1=AffinityModelData[AffinityModelData['pchembl_value']>5.0].index
AffinityModelData.loc[id1,'pchembl_cat']=1 # pchembl_value>5.0 active (1) and rest inactive (0)
AffinityModelData.to_csv('Affinity_Modeldata_with2DPCA_classfication.csv')
train_data, test_data = train_test_split(AffinityModelData, test_size=0.2)

In [24]:
(nr_train,nc_train)=train_data.shape
print (train_data.shape)
(nr_test,nc_test)=test_data.shape
print (test_data.shape)
X_train=train_data.iloc[:,0:nc_train-1]
y_train=train_data["pchembl_cat"]
X_test=test_data.iloc[:,0:nc_test-1]
y_test=test_data["pchembl_cat"]

(312, 1063)
(78, 1063)


In [25]:
AffinityModelData.head(4)

,fp_0,fp_1,fp_2,fp_3,fp_4,fp_5,fp_6,fp_7,fp_8,fp_9,...,34,35,36,37,38,39,40,41,pchembl_value,pchembl_cat
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.287789,-0.273479,-0.906460,-0.020014,-0.776637,-0.105636,0.000000,0.000000,4.82,0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.291834,-0.272189,-0.898900,0.014533,-0.729390,-0.340343,0.000000,0.000000,4.82,0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.273632,-0.091649,-1.124448,-0.269138,-0.779118,-0.178752,-1.935218,2.207942,10.19,1
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.273632,-0.091649,-1.124448,-0.269138,-0.779118,-0.178752,-1.935218,2.207942,10.74,1


In [52]:
s=np.sum(AffinityModelData['pchembl_cat'])
strain=np.sum(y_train)
stest=np.sum(y_test)
print (stest)
print (strain)

68
276


3. Dimensionality Reduction 

In [18]:
# Principal component analysis
ncomps=[150]
var_pca=list()
for ncomp in ncomps:
    pca=PCA(n_components=ncomp)
    scaler=StandardScaler()
    PCAModel= make_pipeline(scaler,pca)
    PCAModel.fit(X_train)
    print(sum(PCAModel.named_steps['pca'].explained_variance_ratio_))
    var_pca.append(sum(PCAModel.named_steps['pca'].explained_variance_ratio_))
Xpcatrain=PCAModel.transform(X_train)
Xpcatest=PCAModel.transform(X_test)

0.9671637861695067


In [19]:
# Truncated SVD Method
svd=TruncatedSVD(n_components=100)
svd.fit(X_train)
Xsvdtrain=svd.transform(X_train)
Xsvdtest=svd.transform(X_test)
print (sum(svd.explained_variance_ratio_))
var_svd=sum(svd.explained_variance_ratio_)

0.966978306461205


4. Regression Tree for Classification

In [123]:
# Training on original data
Xtrain=X_train
ytrain=y_train
Xtest=X_test
ytest=y_test
parameters={'max_depth':[5,10,20]}
regtree=DecisionTreeClassifier()
regtree=GridSearchCV(cv=5,estimator=regtree,param_grid=parameters,refit=True,n_jobs=-1)
T=regtree.fit(Xtrain,ytrain)

ytrainfit=regtree.predict(Xtrain)
ypredtest=regtree.predict(Xtest)

train_cm_regtree=confusion_matrix(ytrain,ytrainfit)

test_cm_regtree=confusion_matrix(ytest,ypredtest)


print ("Training Confusion Matrix =",train_cm_regtree," \n Test Confusion Matrix=",test_cm_regtree)
best_tree_model=regtree.best_estimator_
print (best_tree_model)


Training Confusion Matrix = [[ 36   0]
 [  0 276]]  
 Test Confusion Matrix= [[10  0]
 [ 0 68]]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


NameError: name 'graphviz' is not defined

In [60]:
# Train Regression Tree on SVD scores
Xtrain=Xsvdtrain
ytrain=y_train
Xtest=Xsvdtest
ytest=y_test
parameters={'max_depth':[5,10,20,50]}
regtree=DecisionTreeClassifier()
regtree=GridSearchCV(cv=5,estimator=regtree,param_grid=parameters,refit=True,n_jobs=-1)
regtree.fit(Xtrain,ytrain)
ytrainfit=regtree.predict(Xsvdtrain)
ypredtest=regtree.predict(Xsvdtest)

ytrainfit=regtree.predict(Xtrain)
ypredtest=regtree.predict(Xtest)

train_cm_regtree=confusion_matrix(ytrain,ytrainfit)

test_cm_regtree=confusion_matrix(ytest,ypredtest)


print ("Training Confusion Matrix =",train_cm_regtree," \n Test Confusion Matrix=",test_cm_regtree)
best_tree_model=regtree.best_estimator_
print (best_tree_model)

Training Confusion Matrix = [[ 31   5]
 [  1 275]]  
 Test Confusion Matrix= [[ 1  9]
 [ 4 64]]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


5. XGBoost Classification

In [62]:
from sklearn.ensemble import GradientBoostingClassifier
Xtrain=X_train
ytrain=y_train
Xtest=X_test
ytest=y_test
parameters={'max_depth':[5,10,20],'n_estimators':[10,50,100,150]}
xgboost=GradientBoostingClassifier()
xgboostMdl=GridSearchCV(cv=5,estimator=xgboost,param_grid=parameters,refit=True,n_jobs=-1)
xgboostMdl.fit(Xtrain,ytrain)
ytrainfit=xgboostMdl.predict(Xtrain)
ypredtest=xgboostMdl.predict(Xtest)

train_cm_xgb=confusion_matrix(ytrain,ytrainfit)

test_cm_xgb=confusion_matrix(ytest,ypredtest)


print ("Training Confusion Matrix =",train_cm_xgb," \n Test Confusion Matrix=",test_cm_xgb)


print ("Training R2=",trainR2_xg," Test R2=",testR2_xg)
best_xgboost_Mdl=xgboostMdl.best_estimator_
print (best_xgboost_Mdl)
best_cv_score_xg=xgboostMdl.best_score_


Training Confusion Matrix = [[ 36   0]
 [  0 276]]  
 Test Confusion Matrix= [[10  0]
 [ 0 68]]
Training R2= 1.0  Test R2= 1.0
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=10,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


6. Random Forest Classification

In [63]:
Xtrain=X_train
ytrain=y_train
Xtest=X_test
ytest=y_test
parameters={'max_depth':[5,10,20],'n_estimators':[10,50,100,150]}
RF=RandomForestClassifier()
RFMdl=GridSearchCV(cv=5,estimator=RF,param_grid=parameters,refit=True,n_jobs=-1)
RFMdl.fit(Xtrain,ytrain)
ytrainfit=RFMdl.predict(Xtrain)
ypredtest=RFMdl.predict(Xtest)

train_cm_rf=confusion_matrix(ytrain,ytrainfit)

test_cm_rf=confusion_matrix(ytest,ypredtest)


print ("Training Confusion Matrix =",train_cm_rf," \n Test Confusion Matrix=",test_cm_rf)

best_RF_Mdl=RFMdl.best_estimator_
print (best_RF_Mdl)
best_cv_score_RF=RFMdl.best_score_


Training Confusion Matrix = [[ 34   2]
 [  0 276]]  
 Test Confusion Matrix= [[ 1  9]
 [ 2 66]]
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


/home/shyam_gsdk/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


7. Neural network classification using Tensorflow

In [111]:
from tensorflow.keras import metrics
def build_model(nlayer,n,input_size):
    model=keras.Sequential()
    model.add(layers.Dense(n[0],input_shape=(input_size,),kernel_initializer='glorot_uniform'))
    for k in range(nlayer):
        model.add(layers.Dense(n[k],activation='relu',kernel_initializer='glorot_uniform'))
        model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1,activation='sigmoid'))
    
    opt=tf.keras.optimizers.Adam()
    model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    return model

In [115]:
tf.keras.backend.clear_session()
Xtrain=X_train.values
#Xtrain=Xsvdtrain 
Xtest=X_test.values
#Xtest=Xsvdtest
ytest=y_test.values
ytrain=y_train.values
nlayer=1
n_neurons=10*np.ones(shape=(nlayer,1))
eps=0.001
n_epoch=300
n_batch=20
min_epoch=100
checkpoint_path = "training_11/cp.ckpt"

TFmodel=build_model(nlayer,n_neurons,Xtrain.shape[1])
stop_call = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max',min_delta=0.000001,patience=min_epoch,verbose=1,restore_best_weights=True)
check_pt = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_best_only=True,save_weights_only=True,verbose=0)
modelfit=TFmodel.fit(Xtrain,ytrain,epochs=n_epoch, batch_size=n_batch, validation_split = 0.2, verbose=2,callbacks=[stop_call,check_pt])


Train on 249 samples, validate on 63 samples
Epoch 1/300
249/249 - 0s - loss: 0.3514 - accuracy: 0.8916 - val_loss: 0.4661 - val_accuracy: 0.8413
Epoch 2/300
249/249 - 0s - loss: 0.3104 - accuracy: 0.8956 - val_loss: 0.3976 - val_accuracy: 0.8413
Epoch 3/300
249/249 - 0s - loss: 0.2717 - accuracy: 0.8996 - val_loss: 0.3960 - val_accuracy: 0.8413
Epoch 4/300
249/249 - 0s - loss: 0.2481 - accuracy: 0.8956 - val_loss: 0.3957 - val_accuracy: 0.8413
Epoch 5/300
249/249 - 0s - loss: 0.2297 - accuracy: 0.8996 - val_loss: 0.3920 - val_accuracy: 0.8413
Epoch 6/300
249/249 - 0s - loss: 0.2070 - accuracy: 0.9116 - val_loss: 0.3766 - val_accuracy: 0.8413
Epoch 7/300
249/249 - 0s - loss: 0.1825 - accuracy: 0.9277 - val_loss: 0.3823 - val_accuracy: 0.8571
Epoch 8/300
249/249 - 0s - loss: 0.1642 - accuracy: 0.9478 - val_loss: 0.3763 - val_accuracy: 0.8571
Epoch 9/300
249/249 - 0s - loss: 0.1443 - accuracy: 0.9438 - val_loss: 0.3762 - val_accuracy: 0.8571
Epoch 10/300
249/249 - 0s - loss: 0.1331 - acc

Epoch 81/300
249/249 - 0s - loss: 0.0090 - accuracy: 0.9960 - val_loss: 0.7443 - val_accuracy: 0.8889
Epoch 82/300
249/249 - 0s - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.6778 - val_accuracy: 0.8889
Epoch 83/300
249/249 - 0s - loss: 0.0148 - accuracy: 0.9960 - val_loss: 0.7899 - val_accuracy: 0.8889
Epoch 84/300
249/249 - 0s - loss: 0.0105 - accuracy: 0.9960 - val_loss: 0.7551 - val_accuracy: 0.8889
Epoch 85/300
249/249 - 0s - loss: 0.0106 - accuracy: 0.9960 - val_loss: 0.7347 - val_accuracy: 0.8889
Epoch 86/300
249/249 - 0s - loss: 0.0119 - accuracy: 0.9960 - val_loss: 0.6610 - val_accuracy: 0.9048
Epoch 87/300
249/249 - 0s - loss: 0.0191 - accuracy: 0.9960 - val_loss: 0.6504 - val_accuracy: 0.9048
Epoch 88/300
249/249 - 0s - loss: 0.0065 - accuracy: 0.9960 - val_loss: 0.7433 - val_accuracy: 0.8889
Epoch 89/300
249/249 - 0s - loss: 0.0066 - accuracy: 0.9960 - val_loss: 0.6962 - val_accuracy: 0.9048
Epoch 90/300
249/249 - 0s - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.6942 - v

In [117]:
ypred_test_tf = TFmodel.predict(Xtest)
ypred_train_tf=TFmodel.predict(Xtrain)

ypred_test_tf_cat=np.zeros(ypred_test_tf.shape)
ypred_train_tf_cat=np.zeros(ypred_train_tf.shape)

idtrain=ypred_train_tf>=0.5
idtest=ypred_test_tf>=0.5

ypred_test_tf_cat[idtest]=1

ypred_train_tf_cat[idtrain]=1


confusion_mat_train=confusion_matrix(ytrain,ypred_train_tf_cat)
print (confusion_mat_train)

confusion_mat_test=confusion_matrix(ytest,ypred_test_tf_cat)
print (confusion_mat_test)

[[ 30   6]
 [  0 276]]
[[ 3  7]
 [ 2 66]]
